In [1]:
import os
import sys
### Append the project root directory to sys.path
sys.path.append(os.path.abspath("/Users/prashal/dev/Face-Detector"))

from keras.optimizers import Adam

from models.keras_ssd7 import build_model
from keras_loss_function.keras_ssd_loss import SSDLoss
from misc_utils.utils import create_virtual_folder, format_labels

##### Define model configuration parameters

In [2]:
img_height = 300
img_width = 480
img_channels = 3
intensity_mean = 127.5  
intensity_range = 127.5 
n_classes = 5
min_scale = 0.07
max_scale = 0.9
aspect_ratios = [0.5, 1.0, 2.0]
two_boxes_for_ar1 = True
steps = None
offsets = None
clip_boxes = False
variances = [1.0, 1.0, 1.0, 1.0]
normalize_coords = True

##### Build the keras model

In [3]:
ssd = build_model(
    image_size=(img_height, img_width, img_channels),
    n_classes=n_classes,
    mode="training",
    l2_regularization=0.0005,
    min_scale=min_scale,
    max_scale=max_scale,
    aspect_ratios_global=aspect_ratios,
    aspect_ratios_per_layer=None,
    two_boxes_for_ar1=two_boxes_for_ar1,
    steps=steps,
    offsets=offsets,
    clip_boxes=clip_boxes,
    variances=variances,
    normalize_coords=normalize_coords,
    subtract_mean=intensity_mean,
    divide_by_stddev=intensity_range,
)

adam = Adam(learning_rate=0.001)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
ssd.compile(optimizer=adam, loss=ssd_loss.compute_loss)

##### Set up the data generators

In [4]:
### Create symlinks to train-set
src_folder_train = os.path.abspath("../wider_face/data/WIDER_train/images")
virtual_folder_train = os.path.abspath("../dataset/train/images")
create_virtual_folder(src_folder_train, virtual_folder_train)
### Format training set ground truth labels
train_labels_file = os.path.abspath("../wider_face/data/wider_face_split/wider_face_train_bbx_gt.txt")
formatted_train_labels_file = os.path.abspath("../dataset/train/train_bbox.csv")
format_labels(train_labels_file, formatted_train_labels_file)

'Formatted file succesfully saved to /Users/prashal/dev/Face-Detector/dataset/train/train_bbox.csv'

In [5]:
### Create symlinks to validation-set
src_folder_val = os.path.abspath("../wider_face/data/WIDER_val/images")
virtual_folder_val = os.path.abspath("../dataset/val/images")
create_virtual_folder(src_folder_val, virtual_folder_val)
### Format validation set ground truth labels
val_labels_file = os.path.abspath("../wider_face/data/wider_face_split/wider_face_val_bbx_gt.txt")
formatted_val_labels_file = os.path.abspath("../dataset/val/val_bbox.csv")
format_labels(val_labels_file, formatted_val_labels_file)

'Formatted file succesfully saved to /Users/prashal/dev/Face-Detector/dataset/val/val_bbox.csv'

In [6]:
from data_generator.object_detection_2d_data_generator import DataGenerator

train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

### Images
images_dir_train = virtual_folder_train
images_dir_val = virtual_folder_val

### Ground truth
formatted_train_labels_file
formatted_val_labels_file

'/Users/prashal/dev/Face-Detector/dataset/val/val_bbox.csv'